In [5]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, DensityMatrix, partial_trace
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, QuantumError
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import LogLocator
import os

def find_optimal_n(populations, n_values_list):
    real_pops = [float(pop.real) for pop in populations]
    if not real_pops:
        return 0, 0
    max_pop = max(real_pops)
    tol = 1e-5
    optimal_indices = [i for i, pop in enumerate(real_pops) if abs(pop - max_pop) <= tol]
    if not optimal_indices:
         return n_values_list[0], real_pops[0]
    return n_values_list[max(optimal_indices)], max_pop

def conditional_x_noise(p):
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    P0 = np.array([[1, 0], [0, 0]])
    P1 = np.array([[0, 0], [0, 1]])
    K1 = np.sqrt(1 - p) * np.kron(I, I)
    K2 = np.sqrt(p) * (np.kron(I, P0) + np.kron(X, P1))
    return Kraus([K1, K2])

def apply_global_depolarizing(rho, p):
    n = rho.num_qubits
    dim = 2 ** n
    identity = np.eye(dim) / dim
    return DensityMatrix(rho.data + p * (identity - rho.data))

p_init = 0.85
rho_single = DensityMatrix(np.diag([p_init, 1 - p_init]))

def build_U(n):
    qc = QuantumCircuit(n)
    if n == 1:
        return qc
    qc.x(n-1)
    for i in range(n-2, -1, -1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    if n > 1:
        qc.mcx(list(range(n-1)), n-1)
    qc.x(n-1)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    qc.x(n-1)
    decomposed_qc = transpile(qc, basis_gates=["cx", "sx", "rz"], optimization_level=0)
    return decomposed_qc

def simulate_physical(n, cnot_noise_level, num_rounds=20):
    p_sx = cnot_noise_level / 10
    p_rz = cnot_noise_level / 10
    
    def sx_noise(p):
        I = np.eye(2, dtype=np.complex128)
        X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
        return Kraus([np.sqrt(1 - p) * I, np.sqrt(p) * X])

    def rz_noise(p):
        I = np.eye(2, dtype=np.complex128)
        Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
        return Kraus([np.sqrt(1 - p) * I, np.sqrt(p) * Z])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(QuantumError(sx_noise(p_sx)), 'sx')
    noise_model.add_all_qubit_quantum_error(QuantumError(rz_noise(p_rz)), 'rz')
    noise_model.add_all_qubit_quantum_error(QuantumError(conditional_x_noise(cnot_noise_level)), 'cx')
    
    current_rho = rho_single
    for _ in range(n-1):
        current_rho = current_rho.tensor(rho_single)
    
    decomposed_U = build_U(n)
    
    for _ in range(num_rounds):
        qc = QuantumCircuit(n)
        qc.set_density_matrix(current_rho)
        qc.compose(decomposed_U, inplace=True)
        qc.save_state()
        
        simulator = AerSimulator(method='density_matrix', noise_model=noise_model)
        tqc = transpile(qc, simulator)
        result = simulator.run(tqc).result()
        new_rho = DensityMatrix(result.data(0)['density_matrix'])
        
        traced = partial_trace(new_rho, [n-1])
        current_rho = rho_single.tensor(traced)
    
    return partial_trace(current_rho, list(range(1, n))).data[0,0]

def simulate_theoretical(n, cnot_noise_level, num_rounds=20):
    current_rho = rho_single
    for _ in range(n-1):
        current_rho = current_rho.tensor(rho_single)
    
    decomposed_U = build_U(n)
    ops = decomposed_U.count_ops()
    cnot_count = ops.get('cx', 0)
    
    p = cnot_noise_level * cnot_count * 0.75
    
    for _ in range(num_rounds):
        qc = QuantumCircuit(n)
        qc.set_density_matrix(current_rho)
        qc.compose(decomposed_U, inplace=True)
        qc.save_state()
        
        simulator = AerSimulator(method='density_matrix')
        tqc = transpile(qc, simulator)
        result = simulator.run(tqc).result()
        new_rho = DensityMatrix(result.data(0)['density_matrix'])
        
        if p > 0:
            new_rho = apply_global_depolarizing(new_rho, p)
        
        traced = partial_trace(new_rho, [n-1])
        current_rho = rho_single.tensor(traced)
        
    return partial_trace(current_rho, list(range(1, n))).data[0,0]

cnot_noise_levels = np.logspace(-3, -9, 7) # 1e-3 ... 1e-9
n_values = range(3, 8) 
n_values_list = list(n_values)
num_rounds = 100

physical_optimal_n = []
theoretical_optimal_n = []
physical_optimal_pop = []
theoretical_optimal_pop = []
all_physical_pops = []
all_theoretical_pops = []

print("Starting Simulation...")
for noise_level in cnot_noise_levels:
    print(f"Processing noise level: {noise_level:.1e}")
    
    physical_pops = []
    for n in n_values:
        pop = simulate_physical(n, noise_level, num_rounds)
        physical_pops.append(pop.real)
    
    theoretical_pops = []
    for n in n_values:
        pop = simulate_theoretical(n, noise_level, num_rounds)
        theoretical_pops.append(pop.real)
        
    all_physical_pops.append(physical_pops)
    all_theoretical_pops.append(theoretical_pops)
    
    physical_opt_n, physical_opt_pop = find_optimal_n(physical_pops, n_values_list)
    theoretical_opt_n, theoretical_opt_pop = find_optimal_n(theoretical_pops, n_values_list)
    
    physical_optimal_n.append(physical_opt_n)
    theoretical_optimal_n.append(theoretical_opt_n)
    physical_optimal_pop.append(physical_opt_pop)
    theoretical_optimal_pop.append(theoretical_opt_pop)

PRL_WIDTH_INCH = 3.375 

plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "font.size": 8,            
    "axes.labelsize": 8,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7,
    "mathtext.fontset": "stix",
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "lines.linewidth": 1.2,
    "lines.markersize": 3.5,
    "axes.linewidth": 0.6,
    "grid.linewidth": 0.5,
    "xtick.major.width": 0.6,
    "ytick.major.width": 0.6,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.top": True,
    "ytick.right": True,
    "pdf.fonttype": 42
})

output_dir = "./"
os.makedirs(output_dir, exist_ok=True)


fig1 = plt.figure(figsize=(PRL_WIDTH_INCH, 4.5)) 

gs1 = gridspec.GridSpec(3, 1, figure=fig1, hspace=0.25, 
                        top=0.97, bottom=0.12, left=0.15, right=0.95)

indices_to_plot = [1, 3, 5] # 对应 1e-4, 1e-6, 1e-8
labels_p = [r'$p=10^{-4}$', r'$p=10^{-6}$', r'$p=10^{-8}$']
panel_labels = ['(a)', '(b)', '(c)']

axes_1 = []

for i, idx in enumerate(indices_to_plot):
    ax = fig1.add_subplot(gs1[i])
    axes_1.append(ax)
    
    phys_data = all_physical_pops[idx]
    theo_data = all_theoretical_pops[idx]
    
    ax.plot(theo_data, n_values_list, 's-', color='tab:orange', label='Theoretical')
    ax.plot(phys_data, n_values_list, 'o--', color='tab:blue', label='Physical')
    
    # 纵轴设置
    ax.set_yticks(n_values_list)
    ax.set_ylim(min(n_values_list)-0.5, max(n_values_list)+0.5)
    ax.tick_params(top=False, right=False, which='both')
    if i == 2:
        p_x = 0.75
        p_ha = 'left'
    else:
        p_x = 0.85 
        p_ha = 'left'
    # 标签和网格
    ax.text(p_x, 0.9, labels_p[i], transform=ax.transAxes, 
            fontsize=8, va='top', ha='left')
    ax.text(0.05, 1.015, panel_labels[i], transform=ax.transAxes, 
            fontsize=9, fontweight='bold', va='bottom', ha='right')
    ax.grid(True, linestyle=':', alpha=0.6)
    if i == 1:
        ax.legend(loc='lower left', bbox_to_anchor=(0.05, 0.2), frameon=False, fontsize=7)


fig1.text(0.05, 0.55, r'Qubit Numbers $n$', va='center', rotation='vertical', fontsize=9)
fig1.text(0.55, 0.05, r'Ground-state Population $P_n$', ha='center', fontsize=9)

fig1_path = os.path.join(output_dir, "Fig1_Vertical_Cooling.pdf")
plt.savefig(fig1_path, bbox_inches='tight')
print(f"Figure 1 saved to {fig1_path}")
plt.close(fig1)

fig2 = plt.figure(figsize=(PRL_WIDTH_INCH, 2.5))

gs2 = gridspec.GridSpec(2, 1, figure=fig2, height_ratios=[1, 1], hspace=0.1,
                        top=0.95, bottom=0.15, left=0.15, right=0.95)

ax2_top = fig2.add_subplot(gs2[0])
ax2_bot = fig2.add_subplot(gs2[1]) 


ax2_top.semilogx(cnot_noise_levels, theoretical_optimal_n, 's-', color='tab:orange', label='Theoretical')
ax2_top.semilogx(cnot_noise_levels, physical_optimal_n, 'o--', color='tab:blue', label='Physical')

ax2_top.set_ylabel(r'Optimal Qubits $n_{\mathrm{opt}}$')
ax2_top.set_yticks(range(3, 8))
ax2_top.grid(True, linestyle=':', alpha=0.6)

ax2_top.text(0.03, 1.015, '(a)', transform=ax2_top.transAxes, fontsize=9, fontweight='bold')

plt.setp(ax2_top.get_xticklabels(), visible=False)


ax2_bot.semilogx(cnot_noise_levels, theoretical_optimal_pop, 's-', color='tab:orange',label='Theoretical')
ax2_bot.semilogx(cnot_noise_levels, physical_optimal_pop, 'o--', color='tab:blue', label='Physical')

ax2_bot.set_ylabel(r'$P_{\max}$')
ax2_bot.set_xlabel(r'Error Probability $p$')
ax2_bot.grid(True, linestyle=':', alpha=0.6)
ax2_bot.text(0.03, 1.015, '(b)', transform=ax2_bot.transAxes, fontsize=9, fontweight='bold')
ax2_bot.legend(loc='lower left', frameon=False, fontsize=7)

ax2_top.set_xlim(1e-9 / 2, 1e-3 * 2) 
ax2_bot.set_xlim(1e-9 / 2, 1e-3 * 2)
for ax in [ax2_top, ax2_bot]:
    ax.tick_params(top=False, right=False, which='both')

locmaj = LogLocator(base=10, numticks=10) 
ax2_bot.xaxis.set_major_locator(locmaj)
ax2_bot.minorticks_off()
ax2_top.minorticks_off()  
fig2_path = os.path.join(output_dir, "Fig2_Optimal_Summary.pdf")
plt.savefig(fig2_path, bbox_inches='tight')
print(f"Figure 2 saved to {fig2_path}")
plt.close(fig2)

Starting Simulation...
Processing noise level: 1.0e-03
Processing noise level: 1.0e-04
Processing noise level: 1.0e-05
Processing noise level: 1.0e-06
Processing noise level: 1.0e-07
Processing noise level: 1.0e-08
Processing noise level: 1.0e-09
Figure 1 saved to ./Fig1_Vertical_Cooling.pdf
Figure 2 saved to ./Fig2_Optimal_Summary.pdf
